In [1]:
using DataFrames, CSV
using JLD
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions
using Tables

In [2]:
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-22


Gurobi.Env(Ptr{Nothing} @0x00000000076a6740, false, 0)

Indices:
$$\begin{aligned}
    i&=\text{index of tyre choice}\\
    j&=\text{index of the current number of consecutive laps}\\
    k&=\text{index of lap number}\\
\end{aligned}$$


In [3]:
I = 3
J = 70
L = 70

70

Data:

$$\begin{aligned}
    c_{i,j}&=\text{real number that specifies the amount of time it takes to finish a lap with tyre choice $i$ in the $j$-th consecutive lap} \\
    \Delta t_{ps}&=\text{real number that specifies the amount of time required for a pit stop} \\
\end{aligned}$$


In [9]:
df = DataFrame(CSV.File("laptimes.csv",header=false));
c = Matrix(df[2:end,2:end]);

Delta_t_ps = 20000;

Decision variables:
$$\begin{aligned}
    t_{ijl} &= \text{binary variable that specifies whether tyre $i$ is chosen for the $j$-th consecutive lap in lap $l$} \\
    y_{l} &= \text{binary variable that specifies whether a pit stop is required in lap $l$} \\
\end{aligned}$$

In [10]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV));
@variable(model, t[1:I,1:J,1:L], Bin);
@variable(model, y[1:L], Bin);

Objective Function:
$$\begin{equation}
    \text{minimize} \displaystyle \sum_{l = 1}^{L} \left( \Delta t_{ps} y_{l} + \sum_{i = 1}^{I} \sum_{j = 1}^{J} c_{ij} t_{ijl} \right) 
\end{equation}$$

In [11]:
@objective(model, Min, sum(Delta_t_ps*y[l] + sum(c[i,j]*t[i,j,l] for i=1:I, j=1:J) for l=1:L));

$$\begin{align}
y_{1} &= 0 && \\
\sum_{i = 1}^{I} t_{i11} &= 1\\
\sum_{i = 1}^{I} \sum_{j = 1}^{J} t_{ijl} &= 1 && \forall l \in 2, \dots, L \\
y_{l} &= \sum_{i = 1}^{I} t_{i1l} && \forall l\in 2, \dots, L \\
t_{ijl} &\leq t_{i,j-1,l-1} && \forall i \in 1, \dots, I, \forall j \in 2, \dots J, \forall l \in 2, \dots, L \\
t_{ijl} &= \{ 0, 1\} && \forall i \in 1, \dots, I, \forall j \in 1, \dots J, \forall l \in 1, \dots, L \\
y_{l} &= \{ 0, 1\} && \forall l \in 2, \dots, L
\end{align}$$

In [12]:
@constraint(model, constraint1[1], y[1] == 0);
@constraint(model, constraint2[1], sum(t[i,1,1] for i = 1:I) == 1);
@constraint(model, constraint3[l in 2:L], sum(t[i,j,l] for i = 1:I,j = 1:J) == 1);
@constraint(model, constraint4[l in 2:L], sum(t[i,1,l] for i = 1:I) == y[l]);
@constraint(model, constraint5[i in 1:I, j in 2:J, l in 2:L], t[i,j,l] <= t[i,j-1,l-1]);

┌ Warning: Axis contains one element: 1. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[1]` instead of `1`.
└ @ JuMP.Containers /home/christopher/.julia/packages/JuMP/UqjgA/src/Containers/DenseAxisArray.jl:173
┌ Warning: Axis contains one element: 1. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[1]` instead of `1`.
└ @ JuMP.Containers /home/christopher/.julia/packages/JuMP/UqjgA/src/Containers/DenseAxisArray.jl:173


In [13]:
optimize!(model);

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 14423 rows, 14770 columns and 43336 nonzeros
Model fingerprint: 0x5ee13ecf
Variable types: 0 continuous, 14770 integer (14770 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+04, 9e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 487 rows and 499 columns
Presolve time: 0.15s
Presolved: 13936 rows, 14271 columns, 42209 nonzeros
Variable types: 0 continuous, 14271 integer (14271 binary)
Found heuristic solution: objective 6351051.2599
Found heuristic solution: objective 6053195.6099

Root relaxation: objective 5.988722e+06, 1532 iterations, 0.10 seconds (0.31 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    59

## Retrieving the results

In [20]:
obj_opt = objective_value(model);
obj_opt

5.988722473109243e6

In [27]:
y_opt = value.(y);
t_opt = value.(t);
CSV.write("OptimalPitStopSchedule.csv",  Tables.table(y_opt), writeheader=false);
CSV.write("SoftTyreUseSchedule.csv",  Tables.table(t_opt[1,:,:]), writeheader=false);
CSV.write("MediumTyreUseSchedule.csv",  Tables.table(t_opt[2,:,:]), writeheader=false);
CSV.write("HardTyreUseSchedule.csv",  Tables.table(t_opt[3,:,:]), writeheader=false);
